In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
import transformers
import sys
import tprompt.utils
from os.path import join
import datasets
from typing import Dict, List
from sklearn.tree import plot_tree
import imodelsx.util
import imodelsx.metrics
import numpy as np
import tprompt.utils
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import torch.cuda
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.base import BaseEstimator, ClassifierMixin
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from typing import Dict, List
from dict_hash import sha256
import joblib
import random
import numpy as np
import sklearn.tree
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder

import imodelsx.treeprompt.stump
import ensembling
from ensembling import PromptHooker, modify_activations

# Load prompt vector

In [ ]:
# hyperparams
checkpoint = 'gpt2'
prompts = [
    " This review of a movie is",
    # " This review of a movie is good",
    # " Positive or Negative? The movie was",
    " The sentiment of the movie was",
    # " The plot of the movie was really",
    " The acting in the movie was",
]
# note: also requires specifying the layer for the hook below


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).eval()

In [ ]:
OUTPUTS_ALL = {}
PROMPT_NUM_GLOBAL = 0


def store_activations(module, inputs, outputs):
    global OUTPUTS_ALL
    global PROMPT_NUM_GLOBAL
    OUTPUTS_ALL[PROMPT_NUM_GLOBAL] = outputs.detach().cpu()
    PROMPT_NUM_GLOBAL += 1
    return outputs * 3  # * 0.01


hook = model.transformer.drop.register_forward_hook(store_activations)
for i, prompt in enumerate(prompts):
    inputs = tokenizer(prompt, return_tensors="pt")
    # hook = model.transformer.h[3].register_forward_hook(change_activations)
    logits_modified = model(**inputs).logits

hook.remove()
assert len(OUTPUTS_ALL) == len(prompts)

# most_probable_tokens = torch.topk(logits_modified, k=10, dim=-1)
# print('\n'.join([tokenizer.decode(x)
#   for x in most_probable_tokens.indices[0, -1]]))
# logits_orig = model(**inputs).logits

In [ ]:
for i in range(len(prompts)):
    print(i, prompts[i], OUTPUTS_ALL[i].shape)

In [ ]:
lens = [len(x) for x in tokenizer(prompts)['input_ids']]
lens

In [ ]:
# avg = torch.concat(tuple(OUTPUTS_ALL.values())).mean(axis=0).unsqueeze(0)
vals = list(OUTPUTS_ALL.values())

# add left zero padding eac

# Evaluate accuracies

In [ ]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=200, replace=False))
# dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
# dset_val = dset_val.select(np.random.choice(
#     len(dset_val), size=100, replace=False))

In [14]:
verbalizer = {0: " Negative.", 1: " Positive."}

m = PromptHooker(
    checkpoint=checkpoint,
    prompts=prompts,
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=None,
    prompt_at_start_or_end='end',
)
m.fit(dset_train["text"], dset_train["label"])
for i, prompt in enumerate(prompts):
    print(i, prompt, '->', m.prompt_accs_[i])


print('------------------------')
m = PromptHooker(
    checkpoint=checkpoint,
    prompts=[prompts[0]],
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=avg,
    prompt_at_start_or_end='end',
)
m.fit(dset_train["text"], dset_train["label"])
print('Averaged ->', m.prompt_accs_[0])

Prompt 0:  This review of a movie is


Prompt 1:  The sentiment of the movie was


Prompt 2:  The acting in the movie was


0  This review of a movie is -> 0.435
1  The sentiment of the movie was -> 0.48
2  The acting in the movie was -> 0.47
------------------------
Prompt 0:  This review of a movie is


                                                                                             9.81it/s]

Averaged -> 0.59
